<a href="https://colab.research.google.com/github/Prashant6306/RAG/blob/main/Retrieval_Augmented_Generation_(RAG)_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install langchain openai sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [2]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [2]:
from langchain.vectorstores import FAISS

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings

In [4]:
from langchain.llms import OpenAI

In [5]:
from langchain.chains import RetrievalQA

In [6]:
from langchain.docstore.document import Document

In [7]:
embedding_model=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

<ipython-input-7-b4854f9415dc>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.

In [8]:
documents=[
    Document(page_content="Socrates was a classical Greek philosopher credited as one of the founders of Western philosophy."),
    Document(page_content="He is an enigmatic figure known mostly through the accounts of classical writers like Plato."),
    Document(page_content="Socrates didn’t write anything himself and is famous for the Socratic method."),
    Document(page_content="He was sentenced to death by drinking hemlock in 399 BC.")
]

In [9]:
vector_store = FAISS.from_documents(documents, embedding_model)

In [11]:
from langchain_community.chat_models import ChatOpenAI

In [19]:
# 4. Create Retriever from Vector Store
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

llm = ChatOpenAI(
    temperature=0.3,
    model_name="gpt-3.5-turbo",  # ✅ For GPT-3.5
)

# 6. Create RetrievalQA Chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
)

In [21]:
# 7. Query the Chain
query = "Who was Socrates?"
result = rag_chain(query)

In [22]:
# 8. Print Result
print("Answer:\n", result["result"])
print("\nSources:\n", [doc.page_content for doc in result["source_documents"]])

Answer:
 Socrates was a classical Greek philosopher credited as one of the founders of Western philosophy. He is famous for the Socratic method, a form of cooperative argumentative dialogue to stimulate critical thinking and illuminate ideas.

Sources:
 ['Socrates was a classical Greek philosopher credited as one of the founders of Western philosophy.', 'Socrates didn’t write anything himself and is famous for the Socratic method.']
